# Chapter 4: Design Structure Matrix

Dear user, given an official manual of the product, \
we will compare the official components against the components mentioned in other sources \
in the form of design structure matrixes.

### REQUIREMENTS

For this notebook, you need to have:
- 1 x Pickle file of Scrapped data from an **Official Manual** in _current_ folder (from Chap2.ipynb)
- 4 x Pickle files of Scrapped data from other Online sources in _current_ folder (from Chap2.ipynb)

### TO DO SECTION

In [1]:
'''
Dear user, enter your Product here!
'''

product = "PICO 4 All-in-One VR Headset"

In [2]:
'''
Dear user, enter the directory to the Official Manual here!
'''
manual = "support/_current_/user_manual/user_manual.txt"

In [3]:
'''
Dear user, enter the directories to the 4 other sources here!
'''
youtube_data = "support/_current_/youtube/comments.txt"
reddit_data = "support/_current_/reddit/comments.txt"
pc_gamer_data = "support/_current_/pc_gamer/pc_gamer.txt"
product_desc_data = "support/_current_/product_desc/product_desc.txt"

### RUN AS INTENDED (DO NOT CHANGE ANYTHING.)

In [4]:
! pip install langchain==0.1.9 chromadb==0.4.24 langchain-openai==0.0.8

In [5]:
""" Set up OpenAI API key """
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")       # LangChain requires API key in environment

In [6]:
""" Create DSM folder """
import pandas as pd
search_terms = pd.read_pickle("support/_current_/searchTerms.pkl")

import os
import shutil
try:
    os.makedirs("support/%s/%s" % (search_terms, "DSM"))
except FileExistsError:
    shutil.rmtree('support/%s/DSM' % search_terms)
    os.makedirs("support/%s/%s" % (search_terms, "DSM"))

In [7]:
def initiate_vector_store(file):
    """ Remove 'persist' directory, if any """
    try:
        shutil.rmtree('support/%s/persist' % search_terms)       # remove old version
        print("Deleting previous store")
    except:
        print("No store found")
    
    """ Load Private Documents of User Manual """
    from langchain.document_loaders import TextLoader

    loader = TextLoader(file, encoding='utf-8')
    document = loader.load()

    """ Split Documents into smaller parts """
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    splits = text_splitter.split_documents(document)

    """ Use OpenAI Embeddings """
    from langchain_openai import OpenAIEmbeddings
    embedding = OpenAIEmbeddings()

    """ Create back 'persist' directory """
    persist_directory = 'support/%s/persist' % search_terms     # create new version

    """ Apply embeddings on private documents and save in 'persist' directory """
    from langchain.vectorstores import Chroma

    vectordb = Chroma.from_documents(
        documents=splits,                           # target the splits created from the documents loaded
        embedding=embedding,                        # use the OpenAI embedding specified
        persist_directory=persist_directory         # store in the persist directory for future use
    )

    vectordb.persist()                              # store vectordb

### Getting Official Components from User Manual

##### Query ChatGPT for the Official Components

In [8]:
def private_query(file, question, save):
    """ Retrieve vectordb created """
    import pandas as pd
    search_terms = pd.read_pickle("support/_current_/searchTerms.pkl")

    from langchain_openai import OpenAIEmbeddings
    embedding = OpenAIEmbeddings()

    from langchain.vectorstores import Chroma
    persist_directory = 'support/%s/persist' % search_terms

    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
        )

    print("Processing folder:", search_terms)
    print("Size of Vector Database", vectordb._collection.count())    # same as before
    
    """ Apply language model and vectordb for Chatbot """
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)
                    
    from langchain.chains import RetrievalQA

    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 15, "fetch_k": 10}),
        # retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 4}),
        # retriever=vectordb.as_retriever(),
        return_source_documents=True
        )
    
    """ Ready to use GPT """
    template = " Express the answer only as a Python list containing a short non-descriptive name of the physical component. If you don't know the answer to the question, just come up with the components."

    prompt = question + template
    result = qa_chain({"query": prompt})

    components = result["result"]
    print("Components:", components)

    import pickle
    pickle.dump(components, open("support/%s/DSM/%s.pkl" % (search_terms, save), "wb"))

In [9]:
question = f"Identify ten key physical components for {product}."
initiate_vector_store(manual)
private_query(manual, question, "official_components")

Deleting previous store
Processing folder: PICO 4 All-in-One VR Headset
Size of Vector Database 23


c:\Users\tanhu\AppData\Local\Programs\Python\Python39\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Components: ["VR Headset", "Controllers", "1.5V AA Alkaline Batteries", "Glasses Spacer", "Nose Pad", "Controller Lanyards", "USB-C Power Adapter", "USB-C to C 2.0 Data Cable", "Face Cushion", "Top Strap"]


In [10]:
question = f"Identify ten most mentioned physical components for {product}."
initiate_vector_store(youtube_data)
private_query(youtube_data, question, "youtube")

No store found
Processing folder: PICO 4 All-in-One VR Headset
Size of Vector Database 2025
Components: ["VR Headset", "Controllers", "Batteries", "Glasses Spacer", "Nose Pad", "Controller Lanyards", "USB-C Power Adapter", "USB-C to C 2.0 Data Cable", "Face Cushion", "Top Strap"]


In [11]:
question = f"Identify ten most mentioned physical components for {product}."
initiate_vector_store(reddit_data)
private_query(reddit_data, question, "reddit")

No store found
Processing folder: PICO 4 All-in-One VR Headset
Size of Vector Database 3741
Components: ["headstrap", "face shield", "face cushion", "top strap", "metal buckle", "foam", "pro controllers", "display", "audio", "hardware"]


In [12]:
question = f"Identify ten most mentioned physical components for {product}."
initiate_vector_store(pc_gamer_data)
private_query(pc_gamer_data, question, "pc_gamer")

No store found
Processing folder: PICO 4 All-in-One VR Headset
Size of Vector Database 3757
Components: ["headset", "glasses spacer", "wheel adjustment", "nose pad", "headband", "face shield", "foam", "controllers", "passthrough control", "audio"]


In [13]:
question = f"Identify ten most mentioned physical components for {product}."
initiate_vector_store(product_desc_data)
private_query(product_desc_data, question, "product_desc")

No store found
Processing folder: PICO 4 All-in-One VR Headset
Size of Vector Database 3766
Components: ["Fast-LCD screens", "headset", "controllers", "settings menu", "optical sensors", "nose pad", "headband", "wheel adjustment", "glasses spacer", "infrared optical positioning system"]


In [14]:
def auto_DSM(file):
    import pickle
    import pandas as pd
    from openai import OpenAI
    import numpy as np
    import matplotlib.pyplot as plt
    import json

    try:
        # Load official components data
        official_components_data = pd.read_pickle(f"support/{search_terms}/DSM/official_components.pkl")
        file_data = pd.read_pickle(f"support/{search_terms}/DSM/{file}.pkl")

        client = OpenAI()

        question = f"Given 2 lists of physical components, {official_components_data} and {file_data}, design a Design Structure Matrix with 1s and 0s to compare the physical components."
        template = "  Express the answer only as a Python dictionary (with double quotes, strictly no single quotes). The first key should be empty with the value as the components from the first list. the key as a compared component from the second list (do not add numbering) and the value as the 1s and 0s. Do not add additional comments. Only output the dictionary."
        prompt = question + template

        chat_completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,  # adjust persona and temperature
        )

        result = chat_completion.choices[0].message.content
        print(result)  # Print the result

        data = json.loads(result)

        components = data[""]
        del data[""]  # Remove the first key since it represents an empty string

        # Construct DSM matrix
        num_components = len(components)
        dsm_matrix = np.zeros((num_components, num_components), dtype=int)

        for i, component in enumerate(components):
            for j, compared_component in enumerate(components):
                dsm_matrix[i, j] = data.get(component, [0]*num_components)[j]

        # Display DSM
        plt.imshow(dsm_matrix, cmap='viridis', interpolation='nearest')
        plt.xticks(np.arange(num_components), components, rotation=45, ha='right')
        plt.yticks(np.arange(num_components), components)
        plt.title("Design Structure Matrix")
        plt.show()

    except Exception as e:
        print("Invalid Response. Please regenerate.")
        print(e)  # Print the exception for debugging purposes



In [15]:
def auto_DSM(file):
    import pickle
    import pandas as pd
    from openai import OpenAI
    import numpy as np
    import json

    try:
        # Load official components data
        official_components_data = pd.read_pickle(f"support/{search_terms}/DSM/official_components.pkl")
        file_data = pd.read_pickle(f"support/{search_terms}/DSM/{file}.pkl")

        client = OpenAI()

        question = f"Given 2 lists of physical components, {official_components_data} and {file_data}, design a Design Structure Matrix with 1s and 0s to compare the physical components."
        template = "  Express the answer only as a Python dictionary (with double quotes, strictly no single quotes). The first key should be empty with the value as the components from the first list. the key as a compared component from the second list (do not add numbering) and the value as the 1s and 0s. Do not add additional comments. Only output the dictionary."
        prompt = question + template

        chat_completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,  # adjust persona and temperature
        )

        result = chat_completion.choices[0].message.content

        dsm_data = json.loads(result)

        components = dsm_data[""]
        del dsm_data[""]

        # Construct DSM DataFrame
        dsm_df = pd.DataFrame(dsm_data, index=components)

        print("Design Structure Matrix:")
        print(dsm_df)

    except Exception as e:
        print("Invalid Response. Please regenerate.")
        print(e)  # Print the exception for debugging purposes


In [16]:
auto_DSM('youtube')

Design Structure Matrix:
                            VR Headset  Controllers  Batteries  \
VR Headset                           1            0          0   
Controllers                          0            1          0   
1.5V AA Alkaline Batteries           0            0          1   
Glasses Spacer                       0            0          0   
Nose Pad                             0            0          0   
Controller Lanyards                  0            0          0   
USB-C Power Adapter                  0            0          0   
USB-C to C 2.0 Data Cable            0            0          0   
Face Cushion                         0            0          0   
Top Strap                            0            0          0   

                            Glasses Spacer  Nose Pad  Controller Lanyards  \
VR Headset                               0         0                    0   
Controllers                              0         0                    0   
1.5V AA Alkaline 

In [17]:
auto_DSM('reddit')

Design Structure Matrix:
                            headstrap  face shield  face cushion  top strap  \
VR Headset                          0            0             0          0   
Controllers                         0            0             0          0   
1.5V AA Alkaline Batteries          0            0             0          0   
Glasses Spacer                      0            0             0          0   
Nose Pad                            0            0             0          0   
Controller Lanyards                 0            0             0          0   
USB-C Power Adapter                 0            0             0          0   
USB-C to C 2.0 Data Cable           0            0             0          0   
Face Cushion                        0            0             1          0   
Top Strap                           1            0             0          1   

                            metal buckle  foam  pro controllers  display  \
VR Headset                   

In [18]:
auto_DSM('pc_gamer')

Design Structure Matrix:
                            headset  glasses spacer  wheel adjustment  \
VR Headset                        1               0                 0   
Controllers                       0               0                 0   
1.5V AA Alkaline Batteries        0               0                 0   
Glasses Spacer                    0               1                 0   
Nose Pad                          0               0                 0   
Controller Lanyards               0               0                 0   
USB-C Power Adapter               0               0                 0   
USB-C to C 2.0 Data Cable         0               0                 0   
Face Cushion                      0               0                 0   
Top Strap                         0               0                 0   

                            nose pad  headband  face shield  foam  \
VR Headset                         0         0            0     0   
Controllers                      

In [19]:
auto_DSM('product_desc')

Design Structure Matrix:
                            Fast-LCD screens  headset  controllers  \
VR Headset                                 0        1            0   
Controllers                                0        0            1   
1.5V AA Alkaline Batteries                 0        0            0   
Glasses Spacer                             0        0            0   
Nose Pad                                   0        0            0   
Controller Lanyards                        0        0            0   
USB-C Power Adapter                        0        0            0   
USB-C to C 2.0 Data Cable                  0        0            0   
Face Cushion                               0        0            0   
Top Strap                                  0        0            0   

                            settings menu  optical sensors  nose pad  \
VR Headset                              0                0         0   
Controllers                             0                0  